## Sindrome gripal

Agregando o dataset *Sindrome Gripal*

In [1]:
# importando bibliotecas do sistema

In [2]:
from functools import reduce
import os

In [3]:
# importando funções e tipos do pyspark

In [4]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [5]:
# importando scripts proprios  [!] nesse caso são minhas configurações [!]

In [6]:
from config import spark

22/04/25 18:36:08 WARN Utils: Your hostname, eniac resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/04/25 18:36:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/25 18:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/25 18:36:08 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
22/04/25 18:36:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
# setando minha pasta pdrão [!] altere aqui onde você salva suas coisas [!]

In [8]:
datalake = '/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE'

In [9]:
version = '2022-04-19/'

In [10]:
# carregando todos os datasets disponiveis do sindrome gripal 2020 - 2021 - 2022

In [11]:
sg20 = spark.read.csv(os.path.join(datalake, version, 'sindrome_gripal-2020'), header=True, sep=';')
sg21 = spark.read.csv(os.path.join(datalake, version, 'sindrome_gripal-2021'), header=True, sep=';')
sg22 = spark.read.csv(os.path.join(datalake, version, 'sindrome_gripal-2022'), header=True, sep=';')

In [12]:
df = reduce(lambda sg, sg1: sg.unionAll(sg1), [sg20, sg21, sg22])

In [13]:
# pegando as colunas que importam p/ melhor visualização

In [14]:
COLUMNS = [
    'dataInicioSintomas',
    'dataColetaTeste1',
    'dataColetaTeste2',
    'dataColetaTeste3',
    'dataColetaTeste4',
    'dataNotificacao',
    'municipioIBGE',
    'municipioNotificacaoIBGE',
    'evolucaoCaso',
    'classificacaoFinal'
]

In [15]:
df = df.select(COLUMNS)

In [16]:
# criando a coluna data a partir das colunas dataInicioSintomas, dataColetaTeste4, dataColetaTeste3... 
# em ultimo caso a data sera preenchida com dataNotificacao

In [17]:
df = df.withColumn(
    'date',
    F.when(
        F.col('dataInicioSintomas').isNotNull(),
        F.lit(F.col('dataInicioSintomas'))
    ).when(
        F.col('dataInicioSintomas').isNull(),
        F.when(
            F.col('dataColetaTeste4').isNotNull(),
            F.lit(F.col('dataColetaTeste4'))
        ).when(
            F.col('dataColetaTeste4').isNull(),
            F.when(
                F.col('dataColetaTeste3').isNotNull(),
                F.lit(F.col('dataColetaTeste3'))
            ).when(
                F.col('dataColetaTeste3').isNull(),
                F.when(
                    F.col('dataColetaTeste2').isNotNull(),
                    F.lit(F.col('dataColetaTeste2'))
                ).when(
                    F.col('dataColetaTeste2').isNull(),
                    F.lit(F.col('dataColetaTeste1'))
                ).otherwise(F.col('dataNotificacao'))
            )
        )
    )
)

In [18]:
# preenchendo codigos de residencia faltantes com os de notificacao

In [19]:
df = df.withColumn(
    'municipioIBGE',
    F.when(
        F.col('municipioIBGE').isNull(),
        F.lit(F.col('municipioNotificacaoIBGE'))
    ).otherwise(F.col('municipioIBGE'))
)

In [20]:
df = df.withColumnRenamed('municipioIBGE', 'code_mun_ibge')

In [21]:
# criando algumas colunas mais simples

In [22]:
df = df.withColumn('newCases_SG', F.when(F.col('classificacaoFinal').startswith('Conf'), F.lit(1)))
df = df.withColumn('new_des_SG', F.when(F.col('classificacaoFinal') == 'Descartado', F.lit(1)))
df = df.withColumn('new_undefined_SG', F.when(F.col('classificacaoFinal').isNull(), F.lit(1)))

In [23]:
# óbitos [ covid - descartados - nao definidos ]

In [24]:
df = df.withColumn(
    'newDeath_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Óbito'), F.lit(1)
    )
).withColumn(
    'newDeath_des_SG',
    F.when(
        (F.col('new_des_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Óbito'), F.lit(1)
    )
).withColumn(
    'newDeath_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Óbito'), F.lit(1)
    )
)

In [25]:
# recuperados [ covid - descartados - nao definidos ]

In [26]:
df = df.withColumn(
    'new_recovered_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Cura'), F.lit(1)
    )
).withColumn(
    'new_recovered_des_SG',
    F.when(
        (F.col('new_des_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Cura'), F.lit(1)
    )
).withColumn(
    'new_recovered_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Cura'), F.lit(1)
    )
)

In [27]:
# uti [ covid - descartados - nao definidos ]

In [28]:
df = df.withColumn(
    'hosp_uti_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Internado em UTI'),
        F.lit(1)
    )
).withColumn(
    'hosp_uti_des_SG',
    F.when(
        (F.col('new_des_sg').isNotNull()) &
        (F.col('evolucaoCaso') == 'Internado em UTI'),
        F.lit(1)
    )
).withColumn(
    'hosp_uti_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Internado em UTI'),
        F.lit(1)
    )
)

In [29]:
# tratamento domiciliar [ covid - descartados - nao definidos ]

In [30]:
df = df.withColumn(
    'dom_trat_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Em tratamento domiciliar'),
        F.lit(1)
    )
).withColumn(
    'dom_trat_des_SG',
    F.when(
        (F.col('new_des_sg').isNotNull()) &
        (F.col('evolucaoCaso') == 'Em tratamento domiciliar'),
        F.lit(1)
    )
).withColumn(
    'dom_trat_undefined_SG',
    F.when(
        (F.col('new_undefined_sg').isNotNull()) &
        (F.col('evolucaoCaso') == 'Em tratamento domiciliar'),
        F.lit(1)
    )
)

In [31]:
# clinico [ covid - descartados - nao definidos ]

In [32]:
df = df.withColumn(
    'hos_clinc_SG',
    F.when(
        (F.col('newCases_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Internado'),
        F.lit(1)
    )
).withColumn(
    'hos_clinc_des_SG',
    F.when(
        (F.col('new_des_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Internado'),
        F.lit(1)
    )
).withColumn(
    'hos_clinc_undefined_SG',
    F.when(
        (F.col('new_undefined_SG').isNotNull()) &
        (F.col('evolucaoCaso') == 'Internado'),
        F.lit(1)
    )
)

In [33]:
# essas colunas foram mapeadas só pra facilitar o processo de agregação

In [34]:
new_columns = [
    'newCases_SG', 'new_des_SG', 'new_undefined_SG',
    'newDeath_SG', 'newDeath_des_SG', 'newDeath_undefined_SG',
    'new_recovered_SG', 'new_recovered_des_SG', 'new_recovered_undefined_SG',
    'hosp_uti_SG', 'hosp_uti_des_SG', 'hosp_uti_undefined_SG',
    'dom_trat_SG', 'dom_trat_des_SG', 'dom_trat_undefined_SG',
    'hos_clinc_SG', 'hos_clinc_des_SG', 'hos_clinc_undefined_SG',

]

In [35]:
df.groupby('date', 'code_mun_ibge').agg(
    *[F.count(col).alias(col) for col in new_columns]
).filter(F.col('date') >= '2020-01-01').orderBy('date') \
.write.csv(os.path.join(datalake, version, 'preprocess', 'sindrome_gripal'), header=True, mode='overwrite')